In [2]:
# Import libraries
import random
import requests
import json

In [3]:
def decimal_to_binary(value, num_bits=8):
    """
    Convert a decimal integer value to its binary representation with a specified number of bits.

    args:
        value (int): The decimal value to be converted to binary. Ex. 10
        num_bits (int): The number of bits in the resulting binary number. Default value is 8.

    Returns:
        str: The binary representation of the decimal value, padded with zeroes to the specified number of bits. Ex. '00001010'
    """
    # Convert the decimal value to binary and remove the '0b' prefix
    binary_value = bin(value)[2:]

    # Pad the binary value with zeroes to the specified number of bits
    padded_binary_value = binary_value.zfill(num_bits)

    return padded_binary_value

In [4]:
def binary_to_decimal(value):
    """
    Convert a binary value to its decimal representation.

    args:
        value (str): The binary value to be converted to decimal. Ex. '00001010'

    Returns:
        int: The decimal representation of the binary value. Ex. 10
    """
    return int(value, 2)

In [5]:
def generate_chromosome(chromosome_size):
    """
    Generate a chromosome consisting of genes with random values from 0 to 255.

    args:
        chromosome_size (int): The number of genes in the chromosome. Ex. 3

    Returns:
        list: A list of random integers from 0 to 255. Ex. [10, 20, 30]
    """
    return [random.randint(0, 255) for _ in range(chromosome_size)]

In [6]:
def generate_population(population_size, chromosome_size):
    """
    Generate a population of chromosomes of a given size.

    args:
        population_size (int): The number of inividuals in the population.
        chromosome_size (int): The number of chromosomes in each individual.

    Returns:
        list: A list of inividuals, each consisting of chromosomes with random values from 0 to 255.
    """
    return [generate_chromosome(chromosome_size) for _ in range(population_size)]

In [7]:
def crossover(best, population):
    """
    Perform one-point crossover between the best indivual and a population of individuals.
    To do this, the genes are converted to binary, and a random crossover point is selected.
    """

    # Convert the genes to binary
    best_binary = [decimal_to_binary(gene) for gene in best]
    population_binary = [
        [decimal_to_binary(gene) for gene in individual] for individual in population
    ]

    # Perform one-point crossover for each gene in the chromosome
    crossover_population = []
    for individual in population_binary:
        crossover_individual = []
        for best_gene, individual_gene in zip(best_binary, individual):
            # Select a random crossover point
            crossover_point = random.randint(0, len(best_gene))
            # Perform crossover
            crossover_gene = (
                best_gene[:crossover_point] + individual_gene[crossover_point:]
            )
            crossover_individual.append(crossover_gene)
            # print(f"Best: {best_gene} | Random: {individual_gene} | Crossover: {crossover_point} | Crossover: {crossover_gene}")
        crossover_population.append(crossover_individual)

    # Convert the binary genes back to decimal
    crossover_population = [
        [binary_to_decimal(gene) for gene in individual]
        for individual in crossover_population
    ]
    # print(f"Crossover: {crossover_population}")

    return crossover_population

In [8]:
def mutation(population, mutation_rate=0.05):
    """
    Perform mutation on a population of individuals.
    To do this, the genes are converted to binary, each bit is flipped with a probability of mutation_rate, and the genes are converted back to decimal.
    """

    # Convert the genes to binary
    population_binary = [
        [decimal_to_binary(gene) for gene in individual] for individual in population
    ]
    # print(population_binary)

    # Perform mutation for each gene in the chromosome, flipping each bit with a probability of mutation_rate
    mutated_population = []
    for individual in population_binary:
        mutated_individual = []
        for gene in individual:
            mutated_gene = "".join(
                [
                    str(int(bit) ^ 1) if random.random() < mutation_rate else bit
                    for bit in gene
                ]
            )
            mutated_individual.append(mutated_gene)
            # print(f"Original: {gene} | Mutated: {mutated_gene}")
        mutated_population.append(mutated_individual)

    # Convert the binary genes back to decimal
    mutated_population = [
        [binary_to_decimal(gene) for gene in individual]
        for individual in mutated_population
    ]
    # print(f"Mutated: {mutated_population}")

    return mutated_population

In [9]:
def generate_new_population(population, fitness_values):
    # Select the best individual
    best_index = fitness_values.index(max(fitness_values))
    best_individual = population[best_index]
    print(f"Population: {population}")
    print(f"Fitness: {fitness_values}")
    print(f"Best: {best_individual}")

    # Generate a random population
    population = generate_population(30, 3)
    print(f"Int Population: {population}")

    # Perform crossover between the best individual and each individual in the population
    crossovered_population = crossover(best_individual, population)
    print(f"Crossover: {crossovered_population}")

    # Mutate the crossovered population with a probability of 0.05
    mutated_population = mutation(crossovered_population)
    print(f"Mutated: {mutated_population}")

    # Select 6 individuals from the mutated population
    new_population = random.sample(mutated_population, 6)

    # Replace the best individual in the same position in the new population
    new_population[best_index] = best_individual
    print(f"New Population: {new_population}")

    return new_population

In [10]:
def update_entities(population):

    URL_LIGHT_FIXTURE_1 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:001/attrs?type=light_fixture"
    URL_LIGHT_FIXTURE_2 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:002/attrs?type=light_fixture"
    URL_LIGHT_FIXTURE_3 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:003/attrs?type=light_fixture"

    HEADERS = {
        "Content-Type": "application/json",
        "fiware-service": "vfarm",
        "fiware-servicepath": "/vfarm",
    }

    # Create payload for each light fixture
    payload_1l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[0]}}
    )
    payload_1r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[1]}}
    )
    payload_2l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[2]}}
    )
    payload_2r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[3]}}
    )
    payload_3l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[4]}}
    )
    payload_3r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[5]}}
    )

    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_1, headers=HEADERS, data=payload_1l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_1, headers=HEADERS, data=payload_1r
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_2, headers=HEADERS, data=payload_2l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_2, headers=HEADERS, data=payload_2r
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_3, headers=HEADERS, data=payload_3l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_3, headers=HEADERS, data=payload_3r
    )
    print(response.text)

In [11]:
# 01/02/2024
# Definiação da população inicial de indivíduos aleatórios
population = [
    [128, 46, 211],  # 1L
    [251, 197, 251],  # 1R
    [21, 206, 8],  # 2L
    [129, 143, 213],  # 2R
    [237, 169, 157],  # 3L
    [35, 51, 79],  # 3R
]

In [12]:
# 05/02/2024
population = [
    [128, 46, 211],  # 1L
    [251, 197, 251],  # 1R
    [21, 206, 8],  # 2L
    [129, 143, 213],  # 2R - Best individual
    [237, 169, 157],  # 3L
    [35, 51, 79],  # 3R
]

fitness_values = [25.4, 55.6, 25.0, 59.3, 16.3, 46.5]  # Best index 3

# Nova população de indivídios com cruzamento, mutação e seleção.
new_population = [
    [1, 187, 90],
    [5, 207, 213],
    [129, 143, 213],
    [129, 143, 213],
    [62, 228, 122],
    [30, 246, 132],
]

In [13]:
# 08/02/2024
population = [
    [1, 187, 90],  # 1L
    [5, 207, 213],  # 1R
    [129, 143, 213],  # 2L
    [129, 143, 213],  # 2R
    [62, 228, 122],  # 3L - 2R - Best individual
    [30, 246, 132],  # 3R
]

fitness_values = [53.2, 58.3, 57.2, 62.0, 80.8, 55.6]  # Best index 4

# Nova população de indivídios com cruzamento, mutação e seleção.
new_population = [
    [61, 171, 254],
    [60, 109, 92],
    [45, 230, 117],
    [105, 231, 246],
    [62, 228, 122],
    [63, 163, 81],
]

In [14]:
# 12/02/2024
population = [
    [61, 171, 254],  # 1L - Best individual
    [60, 109, 92],  # 1R
    [45, 230, 117],  # 2L
    [105, 231, 246],  # 2R
    [62, 228, 122],  # 3L
    [63, 163, 81],  # 3R
]

fitness_values = [94.3, 56.2, 58.5, 53.2, 48.2, 25.0]  # Best index 1

# Population: [[61, 171, 254], [60, 109, 92], [45, 230, 117], [105, 231, 246], [62, 228, 122], [63, 163, 81]]
# Fitness: [94.3, 56.2, 58.5, 53.2, 48.2, 25.0]
# Best: [61, 171, 254]
# Int Population: [[33, 230, 37], [127, 176, 38], [130, 123, 84], [136, 205, 48], [162, 163, 120], [21, 35, 58], [9, 40, 157], [65, 36, 112], [13, 55, 99], [6, 140, 148], [199, 129, 131], [209, 95, 26], [254, 108, 189], [220, 153, 61], [66, 232, 28], [7, 182, 180], [136, 173, 117], [144, 52, 132], [243, 53, 238], [246, 100, 51], [48, 226, 104], [242, 237, 7], [159, 151, 123], [149, 247, 101], [98, 92, 3], [57, 90, 133], [109, 22, 112], [194, 164, 133], [191, 237, 195], [222, 5, 10]]
# Crossover: [[33, 171, 229], [61, 176, 166], [58, 187, 212], [60, 141, 254], [34, 171, 248], [53, 171, 250], [61, 168, 157], [57, 164, 254], [61, 175, 227], [62, 140, 252], [39, 129, 195], [17, 175, 254], [126, 171, 253], [60, 169, 253], [2, 168, 220], [39, 182, 254], [56, 173, 253], [60, 171, 228], [61, 181, 238], [54, 100, 255], [60, 162, 254], [58, 173, 7], [31, 171, 254], [149, 171, 245], [34, 171, 227], [57, 186, 254], [45, 170, 254], [61, 164, 253], [63, 173, 227], [62, 171, 138]]
# Mutated: [[225, 171, 229], [61, 176, 166], [27, 185, 209], [60, 141, 254], [34, 163, 250], [53, 162, 251], [61, 160, 157], [57, 164, 254], [61, 171, 227], [62, 142, 252], [166, 129, 211], [1, 175, 254], [126, 203, 253], [60, 169, 253], [2, 168, 204], [39, 182, 190], [24, 169, 253], [60, 171, 236], [61, 177, 238], [50, 100, 255], [60, 162, 190], [58, 173, 5], [31, 171, 250], [149, 171, 245], [42, 171, 227], [57, 56, 250], [45, 234, 126], [57, 164, 253], [63, 173, 227], [62, 171, 138]]
# New Population: [[61, 171, 254], [31, 171, 250], [42, 171, 227], [126, 203, 253], [60, 162, 190], [24, 169, 253]]

new_population = [
    [61, 171, 254],
    [31, 171, 250],
    [42, 171, 227],
    [126, 203, 253],
    [60, 162, 190],
    [24, 169, 253],
]

In [15]:
# 14/02/2024
population = [
    [61, 171, 254],
    [31, 171, 250],
    [42, 171, 227],
    [126, 203, 253],  # Best individual
    [60, 162, 190],
    [24, 169, 253],
]

fitness_values = [71.4, 50.3, 12.0, 74.0, 36.8, 51.5]

new_population = generate_new_population(population, fitness_values)

# Population: [[61, 171, 254], [31, 171, 250], [42, 171, 227], [126, 203, 253], [60, 162, 190], [24, 169, 253]]
# Fitness: [71.4, 50.3, 12.0, 74.0, 36.8, 51.5]
# Best: [126, 203, 253]
# Int Population: [[159, 212, 144], [157, 38, 8], [198, 178, 144], [158, 106, 189], [249, 55, 247], [13, 171, 48], [47, 146, 1], [60, 164, 243], [110, 232, 23], [207, 9, 194], [92, 255, 77], [44, 9, 1], [89, 96, 147], [118, 3, 163], [47, 164, 101], [19, 158, 221], [25, 163, 182], [241, 206, 136], [51, 97, 88], [88, 130, 126], [140, 235, 71], [42, 155, 70], [55, 185, 221], [226, 4, 189], [192, 97, 209], [76, 18, 91], [1, 156, 122], [4, 39, 111], [206, 213, 122], [161, 39, 54]]
# Crossover: [[159, 204, 253], [125, 198, 252], [198, 202, 240], [126, 202, 253], [125, 199, 255], [13, 203, 248], [127, 202, 253], [124, 164, 251], [126, 200, 23], [127, 201, 254], [92, 223, 205], [44, 137, 253], [121, 202, 147], [126, 195, 253], [127, 228, 229], [127, 158, 253], [127, 227, 253], [113, 206, 248], [51, 225, 248], [126, 194, 254], [124, 235, 199], [106, 203, 198], [119, 249, 221], [98, 132, 253], [64, 225, 249], [124, 210, 251], [97, 156, 254], [4, 167, 255], [126, 203, 250], [127, 231, 252]]
# Mutated: [[159, 206, 253], [125, 198, 236], [198, 138, 240], [122, 202, 252], [125, 199, 255], [13, 203, 248], [127, 206, 253], [124, 164, 187], [126, 200, 7], [123, 233, 254], [92, 215, 237], [108, 203, 253], [121, 202, 131], [126, 195, 249], [127, 228, 229], [87, 158, 253], [127, 231, 253], [113, 206, 248], [51, 233, 252], [126, 194, 254], [124, 235, 199], [106, 203, 198], [127, 249, 220], [98, 128, 253], [64, 225, 249], [125, 146, 219], [65, 156, 254], [4, 167, 255], [126, 203, 250], [127, 167, 252]]
# New Population: [[198, 138, 240], [51, 233, 252], [122, 202, 252], [126, 203, 253], [87, 158, 253], [108, 203, 253]]

new_population = [
    [198, 138, 240],
    [51, 233, 252],
    [122, 202, 252],
    [126, 203, 253],
    [87, 158, 253],
    [108, 203, 253],
]

Population: [[61, 171, 254], [31, 171, 250], [42, 171, 227], [126, 203, 253], [60, 162, 190], [24, 169, 253]]
Fitness: [71.4, 50.3, 12.0, 74.0, 36.8, 51.5]
Best: [126, 203, 253]
Int Population: [[61, 84, 199], [21, 20, 17], [83, 229, 148], [233, 193, 212], [55, 162, 7], [80, 204, 232], [59, 250, 227], [195, 142, 207], [66, 181, 58], [190, 126, 228], [160, 160, 107], [118, 205, 193], [8, 65, 13], [196, 91, 191], [166, 140, 51], [186, 187, 210], [21, 59, 132], [194, 240, 238], [22, 97, 24], [108, 232, 211], [247, 214, 145], [152, 232, 72], [129, 147, 18], [31, 104, 216], [251, 121, 78], [233, 116, 154], [101, 36, 3], [13, 174, 103], [136, 183, 155], [0, 30, 25]]
Crossover: [[61, 212, 199], [127, 212, 253], [83, 197, 244], [105, 203, 252], [127, 162, 199], [112, 204, 252], [59, 202, 243], [115, 142, 239], [126, 245, 253], [126, 206, 252], [96, 192, 255], [126, 203, 253], [126, 193, 253], [100, 219, 253], [38, 204, 179], [58, 187, 250], [21, 187, 252], [122, 202, 252], [126, 225, 252], [10

In [16]:
# 17/02/2024
population = [
    [198, 138, 240],
    [51, 233, 252],
    [122, 202, 252],
    [126, 203, 253],
    [87, 158, 253],
    [108, 203, 253],
]

fitness_values = [77.9, 53.2, 13.6, 65.3, 53.4, 57.5]

# Population: [[198, 138, 240], [51, 233, 252], [122, 202, 252], [126, 203, 253], [87, 158, 253], [108, 203, 253]]
# Fitness: [77.9, 53.2, 13.6, 65.3, 53.4, 57.5]
# Best: [198, 138, 240]
# Int Population: [[237, 136, 13], [241, 180, 16], [123, 168, 191], [170, 164, 189], [71, 202, 243], [191, 80, 153], [40, 217, 59], [65, 254, 38], [172, 76, 213], [155, 44, 230], [75, 166, 180], [201, 132, 159], [110, 201, 199], [87, 232, 97], [206, 184, 119], [20, 125, 247], [94, 2, 54], [0, 226, 66], [3, 70, 35], [216, 142, 115], [206, 42, 114], [117, 79, 220], [167, 247, 143], [56, 231, 124], [52, 80, 55], [96, 228, 205], [190, 136, 52], [81, 200, 177], [181, 26, 0], [244, 19, 185]]
# Crossover: [[237, 136, 241], [193, 138, 240], [203, 168, 241], [234, 164, 253], [71, 138, 243], [207, 144, 249], [232, 153, 240], [65, 142, 246], [236, 204, 240], [199, 140, 230], [199, 134, 240], [201, 140, 159], [198, 137, 247], [87, 136, 225], [198, 184, 119], [212, 137, 241], [94, 138, 240], [128, 138, 242], [199, 134, 227], [198, 142, 243], [206, 170, 242], [245, 143, 240], [167, 247, 241], [216, 167, 240], [212, 208, 55], [224, 164, 241], [190, 136, 52], [209, 200, 177], [197, 138, 0], [244, 138, 249]]
# Mutated: [[237, 88, 241], [197, 202, 240], [203, 172, 241], [234, 164, 253], [23, 138, 243], [203, 144, 121], [232, 153, 240], [17, 166, 246], [228, 204, 240], [199, 156, 230], [199, 134, 240], [201, 141, 159], [214, 137, 247], [87, 136, 195], [198, 56, 119], [212, 137, 241], [95, 138, 240], [128, 138, 242], [199, 134, 227], [198, 142, 243], [206, 170, 242], [245, 143, 240], [167, 183, 241], [216, 167, 240], [212, 208, 55], [224, 164, 241], [190, 136, 116], [209, 200, 181], [199, 130, 0], [244, 142, 251]]
# New Population: [[198, 138, 240], [197, 202, 240], [245, 143, 240], [244, 142, 251], [214, 137, 247], [232, 153, 240]]

new_population = [
    [198, 138, 240],
    [197, 202, 240],
    [245, 143, 240],
    [244, 142, 251],
    [214, 137, 247],
    [232, 153, 240],
]

In [17]:
# 20/02/2024
population = [
    [198, 138, 240],
    [197, 202, 240],
    [245, 143, 240],
    [244, 142, 251],
    [214, 137, 247],
    [232, 153, 240],
]

fitness_values = [52.4, 37.9, 47.2, 47.8, 27.5, 7.9]

# Population: [[198, 138, 240], [197, 202, 240], [245, 143, 240], [244, 142, 251], [214, 137, 247], [232, 153, 240]]
# Fitness: [52.4, 37.9, 47.2, 47.8, 27.5, 7.9]
# Best: [198, 138, 240]
# Int Population: [[232, 213, 136], [79, 124, 0], [4, 32, 112], [215, 21, 109], [233, 167, 136], [16, 111, 94], [77, 97, 48], [121, 160, 233], [67, 192, 223], [219, 179, 228], [132, 94, 93], [137, 62, 108], [57, 165, 97], [34, 236, 29], [65, 80, 115], [120, 140, 63], [77, 10, 173], [20, 27, 178], [96, 60, 190], [25, 156, 239], [73, 90, 140], [133, 83, 162], [48, 218, 210], [147, 130, 164], [74, 9, 27], [234, 217, 83], [38, 112, 253], [254, 237, 149], [146, 4, 236], [109, 154, 1]]
# Crossover: [[198, 137, 136], [199, 138, 240], [196, 138, 240], [199, 149, 237], [233, 139, 200], [192, 143, 222], [205, 137, 240], [198, 138, 241], [195, 192, 223], [203, 179, 244], [196, 158, 241], [201, 138, 108], [199, 139, 225], [198, 138, 245], [65, 144, 243], [198, 140, 63], [197, 138, 173], [212, 138, 178], [196, 60, 240], [201, 156, 243], [199, 154, 252], [133, 147, 242], [198, 138, 242], [199, 130, 228], [74, 137, 241], [234, 139, 240], [198, 112, 245], [198, 141, 245], [198, 138, 244], [199, 138, 241]]
# Mutated: [[150, 141, 136], [199, 130, 240], [196, 138, 240], [135, 149, 237], [233, 139, 72], [204, 143, 206], [237, 139, 240], [198, 138, 241], [194, 192, 223], [203, 179, 244], [196, 158, 241], [201, 10, 108], [135, 139, 161], [134, 130, 101], [65, 148, 226], [194, 204, 63], [197, 138, 173], [212, 138, 178], [196, 60, 240], [205, 156, 115], [199, 154, 253], [133, 146, 250], [198, 138, 242], [199, 139, 228], [74, 137, 241], [226, 139, 176], [196, 112, 245], [199, 141, 245], [199, 138, 244], [199, 154, 241]]
# New Population: [[198, 138, 240], [237, 139, 240], [212, 138, 178], [199, 154, 253], [199, 130, 240], [196, 158, 241]]

new_population = [
    [198, 138, 240],
    [237, 139, 240],
    [212, 138, 178],
    [199, 154, 253],
    [199, 130, 240],
    [196, 158, 241],
]

In [18]:
# 23/02/2024

population = [
    [198, 138, 240],
    [237, 139, 240],
    [212, 138, 178],
    [199, 154, 253],
    [199, 130, 240],
    [196, 158, 241],
]

In [19]:
# 23/02/2024

population = [
    [211, 169, 243],
    [206, 208, 249],
    [210, 136, 237],
    [199, 154, 253],
    [206, 171, 113],
    [195, 155, 253],
]

fitness_values = [88.3, 55.2, 0, 64.6, 37.1, 49.8]

# Population: [[211, 169, 243], [206, 208, 249], [210, 136, 237], [199, 154, 253], [206, 171, 113], [195, 155, 253]]
# Fitness: [88.3, 55.2, 0, 64.6, 37.1, 49.8]
# Best: [211, 169, 243]
# Int Population: [[39, 1, 13], [30, 49, 112], [121, 217, 19], [69, 239, 169], [137, 176, 42], [22, 125, 10], [48, 198, 129], [95, 23, 129], [199, 237, 69], [165, 159, 169], [107, 96, 133], [138, 61, 204], [63, 216, 113], [87, 23, 44], [214, 189, 243], [232, 158, 17], [84, 238, 110], [149, 160, 91], [54, 230, 26], [80, 172, 252], [41, 156, 30], [135, 206, 89], [220, 152, 144], [127, 18, 20], [233, 137, 192], [147, 222, 129], [71, 205, 230], [107, 112, 19], [158, 195, 174], [142, 9, 60]]
# Crossover: [[211, 169, 241], [222, 49, 240], [249, 217, 243], [211, 239, 243], [137, 169, 170], [210, 125, 242], [240, 170, 129], [223, 171, 241], [211, 237, 241], [211, 169, 169], [235, 96, 133], [202, 189, 240], [63, 216, 243], [215, 167, 242], [214, 169, 243], [232, 158, 243], [211, 174, 242], [209, 160, 251], [214, 169, 242], [208, 168, 240], [209, 172, 254], [199, 169, 217], [220, 169, 144], [215, 169, 212], [211, 169, 240], [211, 170, 241], [215, 169, 242], [235, 168, 243], [158, 169, 174], [210, 169, 252]]
# Mutated: [[211, 233, 241], [158, 17, 240], [249, 217, 241], [211, 239, 243], [137, 169, 170], [219, 125, 242], [240, 174, 133], [219, 171, 241], [211, 239, 225], [211, 169, 185], [233, 96, 165], [202, 189, 248], [55, 200, 241], [147, 175, 242], [198, 169, 251], [232, 156, 243], [211, 238, 210], [209, 160, 251], [214, 169, 242], [208, 172, 240], [209, 172, 254], [199, 169, 217], [156, 169, 144], [215, 169, 212], [211, 129, 240], [211, 170, 240], [215, 169, 242], [235, 168, 243], [158, 169, 174], [210, 168, 252]]
# New Population: [[211, 169, 243], [215, 169, 212], [211, 169, 185], [211, 233, 241], [209, 160, 251], [158, 17, 240]]

new_population = [
    [211, 169, 243],
    [215, 169, 212],
    [211, 169, 185],
    [211, 233, 241],
    [209, 160, 251],
    [158, 17, 240],
]

In [20]:
# 29/03/2024

population = [
    [211, 169, 243],
    [215, 169, 212],
    [211, 169, 185],
    [211, 233, 241],
    [209, 160, 251],
    [158, 17, 240],
]

fitness_values = [84.5, 83.7, 51.0, 52.3, 25.9, 24.9]

# Population: [[211, 169, 243], [215, 169, 212], [211, 169, 185], [211, 233, 241], [209, 160, 251], [158, 17, 240]]
# Fitness: [84.5, 83.7, 51.0, 52.3, 25.9, 24.9]
# Best: [211, 169, 243]
# Int Population: [[66, 106, 165], [162, 4, 248], [97, 75, 249], [167, 248, 60], [99, 96, 157], [218, 250, 29], [173, 158, 59], [181, 21, 237], [140, 26, 61], [136, 183, 14], [232, 53, 182], [211, 23, 62], [173, 175, 140], [7, 72, 18], [232, 241, 60], [28, 234, 242], [116, 75, 124], [224, 96, 56], [49, 81, 74], [64, 80, 144], [82, 113, 46], [44, 53, 128], [55, 62, 119], [98, 59, 55], [7, 170, 167], [42, 148, 221], [7, 240, 73], [19, 150, 150], [143, 63, 147], [182, 244, 175]]
# Crossover: [[66, 170, 241], [210, 168, 248], [225, 171, 249], [231, 184, 252], [99, 168, 241], [218, 250, 243], [211, 168, 243], [209, 169, 253], [208, 154, 253], [136, 183, 14], [216, 181, 246], [211, 151, 254], [221, 175, 140], [215, 72, 243], [200, 241, 240], [211, 170, 242], [116, 169, 252], [224, 160, 242], [211, 169, 202], [210, 176, 240], [210, 161, 46], [204, 181, 240], [55, 168, 247], [226, 187, 183], [211, 170, 167], [234, 148, 245], [7, 176, 241], [19, 168, 246], [215, 191, 243], [182, 244, 247]]
# Mutated: [[64, 170, 245], [210, 169, 248], [225, 171, 249], [167, 184, 252], [227, 168, 241], [218, 250, 243], [215, 168, 243], [213, 169, 253], [80, 154, 253], [8, 183, 14], [216, 181, 246], [147, 151, 254], [221, 174, 140], [199, 72, 99], [200, 209, 242], [211, 170, 214], [116, 169, 252], [224, 164, 242], [211, 161, 202], [210, 56, 240], [210, 179, 38], [205, 181, 240], [51, 168, 247], [226, 59, 183], [83, 227, 167], [234, 156, 229], [7, 176, 245], [26, 168, 246], [215, 183, 243], [182, 116, 247]]
# New Population: [[211, 169, 243], [218, 250, 243], [51, 168, 247], [210, 179, 38], [205, 181, 240], [200, 209, 242]]

new_population = [
    [211, 169, 243],
    [218, 250, 243],
    [51, 168, 247],
    [210, 179, 38],
    [205, 181, 240],
    [200, 209, 242],
]

In [21]:
# 04/03/2024

population = [
    [211, 169, 243],
    [218, 250, 243],
    [51, 168, 247],
    [210, 179, 38],
    [205, 181, 240],
    [200, 209, 242],
]

fitness_values = [75.0, 41.7, 67.4, 33.6, 32.7, 41.2]

# Population: [[211, 169, 243], [218, 250, 243], [51, 168, 247], [210, 179, 38], [205, 181, 240], [200, 209, 242]]
# Fitness: [75.0, 41.7, 67.4, 33.6, 32.7, 41.2]
# Best: [211, 169, 243]
# Int Population: [[228, 143, 246], [250, 128, 148], [210, 218, 211], [117, 60, 11], [251, 211, 30], [234, 55, 118], [176, 190, 205], [35, 184, 114], [73, 89, 108], [144, 62, 102], [218, 167, 57], [96, 157, 251], [134, 57, 176], [209, 124, 20], [47, 181, 234], [23, 125, 255], [106, 27, 99], [72, 50, 127], [147, 107, 28], [164, 18, 220], [91, 173, 139], [114, 135, 99], [58, 152, 219], [36, 212, 186], [106, 255, 20], [225, 32, 254], [151, 227, 184], [135, 235, 153], [116, 40, 210], [86, 91, 122]]
# Crossover: [[228, 143, 243], [250, 160, 148], [210, 154, 243], [211, 172, 203], [219, 147, 254], [234, 183, 246], [208, 174, 205], [211, 184, 114], [209, 169, 236], [210, 174, 230], [218, 167, 241], [224, 157, 251], [134, 185, 240], [209, 172, 244], [211, 169, 243], [211, 169, 243], [234, 27, 227], [216, 170, 243], [211, 171, 220], [208, 169, 220], [91, 169, 243], [210, 171, 227], [218, 184, 243], [164, 180, 250], [234, 171, 244], [225, 168, 242], [211, 163, 184], [135, 235, 241], [244, 40, 243], [214, 171, 250]]
# Mutated: [[196, 143, 243], [250, 176, 148], [82, 146, 243], [211, 172, 79], [251, 147, 254], [234, 183, 246], [208, 174, 205], [211, 185, 114], [153, 169, 236], [214, 174, 230], [216, 167, 241], [224, 29, 251], [134, 185, 240], [209, 172, 244], [211, 169, 243], [211, 169, 243], [234, 27, 227], [248, 170, 243], [211, 171, 220], [208, 137, 221], [91, 169, 243], [210, 171, 227], [218, 248, 243], [160, 176, 248], [234, 171, 244], [97, 160, 179], [211, 163, 184], [135, 239, 240], [244, 40, 243], [246, 171, 250]]


new_population = [
    [211, 169, 243],
    [209, 172, 244],
    [91, 169, 243],
    [216, 167, 241],
    [234, 171, 244],
    [248, 170, 243],
]

In [22]:
# 06/03/2024

population = [
    [211, 169, 243],
    [209, 172, 244],
    [91, 169, 243],
    [216, 167, 241],
    [234, 171, 244],
    [248, 170, 243],
]

fitness_values = [79.7, 69.9, 76.5, 53.9, 45.6, 68.5]

# Population: [[211, 169, 243], [209, 172, 244], [91, 169, 243], [216, 167, 241], [234, 171, 244], [248, 170, 243]]
# Fitness: [79.7, 69.9, 76.5, 53.9, 45.6, 68.5]
# Best: [211, 169, 243]
# Int Population: [[193, 204, 44], [201, 99, 206], [73, 84, 93], [157, 200, 53], [84, 244, 119], [205, 52, 66], [141, 231, 32], [232, 77, 197], [180, 110, 184], [224, 242, 38], [106, 143, 132], [193, 29, 67], [135, 88, 239], [209, 102, 155], [30, 89, 236], [75, 77, 117], [71, 179, 210], [85, 90, 200], [83, 195, 123], [59, 111, 131], [71, 252, 87], [188, 145, 89], [43, 128, 241], [207, 226, 241], [110, 204, 61], [9, 115, 233], [92, 201, 121], [191, 32, 76], [20, 75, 106], [68, 228, 34]]
# Crossover: [[193, 169, 244], [201, 171, 246], [201, 169, 93], [209, 168, 245], [212, 180, 119], [205, 168, 242], [205, 231, 32], [210, 141, 243], [180, 174, 240], [224, 178, 230], [202, 171, 242], [209, 169, 243], [211, 216, 247], [209, 168, 155], [210, 169, 242], [75, 205, 245], [215, 179, 242], [213, 186, 200], [211, 195, 243], [219, 171, 243], [215, 168, 247], [220, 169, 249], [171, 160, 243], [211, 226, 241], [210, 204, 253], [209, 171, 241], [208, 169, 241], [191, 160, 76], [210, 75, 106], [210, 164, 242]]
# Mutated: [[193, 169, 116], [201, 171, 246], [201, 233, 29], [213, 42, 245], [196, 180, 119], [205, 168, 242], [197, 231, 32], [202, 141, 195], [180, 174, 224], [228, 178, 230], [138, 235, 240], [241, 169, 119], [211, 216, 247], [209, 168, 155], [146, 185, 226], [75, 205, 229], [213, 179, 246], [245, 186, 200], [211, 195, 243], [219, 171, 251], [215, 168, 255], [252, 169, 249], [171, 48, 243], [147, 226, 241], [210, 204, 253], [211, 163, 245], [216, 169, 241], [183, 160, 76], [226, 67, 106], [194, 36, 242]]
# New Population: [[211, 169, 243], [210, 204, 253], [202, 141, 195], [180, 174, 224], [138, 235, 240], [147, 226, 241]]

new_population = [
    [211, 169, 243],
    [210, 204, 253],
    [202, 141, 195],
    [180, 174, 224],
    [138, 235, 240],
    [147, 226, 241],
]

In [23]:
# 09/03/2024

population = [
    [211, 169, 243],
    [210, 204, 253],
    [202, 141, 195],
    [180, 174, 224],
    [138, 235, 240],
    [147, 226, 241],
]

fitness_values = [72.84, 66.29, 62.41, 65.71, 67.50, 48.21]

# Population: [[211, 169, 243], [210, 204, 253], [202, 141, 195], [180, 174, 224], [138, 235, 240], [147, 226, 241]]
# Fitness: [72.84, 66.29, 62.41, 65.71, 67.5, 48.21]
# Best: [211, 169, 243]
# Int Population: [[217, 108, 141], [244, 76, 205], [28, 100, 120], [28, 101, 226], [41, 125, 162], [64, 96, 19], [179, 143, 167], [176, 112, 233], [102, 135, 190], [147, 198, 100], [108, 90, 187], [124, 58, 127], [25, 253, 169], [224, 115, 206], [32, 214, 216], [79, 93, 99], [74, 15, 158], [147, 35, 122], [140, 232, 113], [141, 41, 19], [103, 63, 73], [230, 133, 219], [85, 24, 89], [249, 72, 49], [230, 77, 168], [215, 133, 128], [96, 212, 8], [39, 18, 50], [204, 18, 68], [161, 250, 136]]
# Crossover: [[211, 108, 141], [212, 140, 205], [212, 168, 248], [212, 169, 242], [169, 253, 242], [192, 160, 243], [179, 169, 167], [176, 168, 233], [230, 171, 242], [147, 198, 228], [212, 168, 243], [210, 168, 255], [25, 189, 241], [224, 115, 254], [210, 166, 216], [211, 189, 227], [202, 175, 242], [147, 35, 250], [210, 232, 241], [213, 169, 243], [231, 63, 243], [230, 133, 243], [211, 152, 249], [211, 136, 49], [214, 173, 248], [211, 173, 128], [192, 164, 243], [167, 18, 242], [211, 178, 228], [209, 169, 136]]
# Mutated: [[195, 108, 157], [196, 140, 205], [212, 168, 216], [148, 169, 242], [173, 253, 242], [192, 160, 243], [163, 169, 198], [176, 168, 237], [230, 171, 210], [51, 134, 228], [212, 152, 115], [210, 170, 255], [25, 189, 241], [224, 115, 140], [242, 164, 216], [211, 173, 227], [194, 175, 242], [147, 33, 250], [210, 232, 249], [215, 169, 243], [231, 29, 243], [230, 133, 247], [210, 24, 249], [211, 140, 49], [214, 173, 248], [211, 141, 128], [192, 164, 243], [167, 18, 242], [211, 178, 228], [209, 169, 136]]
# New Population: [[211, 169, 243], [148, 169, 242], [211, 141, 128], [167, 18, 242], [211, 140, 49], [242, 164, 216]]

new_population = [
    [211, 169, 243],
    [148, 169, 242],
    [211, 141, 128],
    [167, 18, 242],
    [211, 140, 49],
    [242, 164, 216],
]

In [25]:
# 14/03/2024

population = [
    [211, 169, 243],
    [148, 169, 242],
    [211, 141, 128],
    [167, 18, 242],
    [211, 140, 49],
    [242, 164, 216],
]

fitness_values = [100.00, 49.70, 61.98, 52.75, 46.78, 53.34]

# Population: [[211, 169, 243], [148, 169, 242], [211, 141, 128], [167, 18, 242], [211, 140, 49], [242, 164, 216]]
# Fitness: [100.0, 49.7, 61.98, 52.75, 46.78, 53.34]
# Best: [211, 169, 243]
# Int Population: [[234, 128, 250], [36, 80, 23], [208, 117, 144], [236, 172, 147], [103, 25, 110], [37, 16, 248], [188, 246, 24], [123, 124, 181], [171, 224, 235], [80, 142, 74], [199, 142, 236], [114, 69, 127], [211, 91, 115], [78, 195, 78], [136, 169, 106], [6, 233, 52], [220, 54, 216], [159, 12, 116], [124, 230, 244], [212, 205, 65], [38, 212, 204], [68, 91, 112], [131, 153, 38], [14, 120, 190], [207, 23, 224], [13, 23, 54], [52, 14, 121], [13, 81, 55], [130, 114, 62], [126, 110, 102]]
# Crossover: [[234, 168, 242], [212, 168, 151], [210, 165, 240], [208, 172, 211], [103, 169, 243], [197, 144, 240], [210, 170, 243], [211, 168, 243], [235, 224, 235], [208, 142, 202], [211, 142, 252], [210, 165, 127], [211, 171, 243], [214, 163, 246], [200, 169, 242], [198, 169, 242], [220, 168, 240], [223, 168, 244], [212, 166, 240], [212, 173, 225], [210, 164, 244], [211, 91, 240], [195, 185, 246], [214, 169, 246], [211, 169, 224], [211, 171, 54], [211, 169, 241], [13, 161, 183], [210, 170, 242], [222, 174, 243]]
# Mutated: [[238, 184, 242], [212, 168, 151], [210, 165, 176], [208, 184, 211], [103, 171, 243], [161, 144, 240], [210, 170, 243], [211, 168, 243], [227, 160, 235], [208, 142, 202], [83, 142, 252], [146, 165, 63], [211, 171, 243], [214, 171, 244], [200, 169, 242], [198, 169, 242], [221, 168, 240], [223, 136, 244], [212, 134, 240], [212, 173, 233], [210, 164, 244], [211, 91, 240], [163, 185, 198], [214, 169, 182], [211, 169, 224], [211, 139, 54], [209, 169, 241], [13, 129, 183], [242, 187, 242], [222, 174, 243]]
# New Population: [[211, 169, 243], [211, 168, 243], [238, 184, 242], [209, 169, 241], [227, 160, 235], [214, 169, 182]]

new_population = [
    [211, 169, 243],
    [211, 168, 243],
    [238, 184, 242],
    [209, 169, 241],
    [227, 160, 235],
    [214, 169, 182],
]


In [ ]:
# 19/03/2024

population = [
    [211, 169, 243],
    [211, 168, 243],
    [238, 184, 242],
    [209, 169, 241],
    [227, 160, 235],
    [214, 169, 182],
]

fitness_values = [100.00, 49.70, 61.98, 52.75, 46.78, 53.34]
